In [10]:
import numpy as np
import pandas as pd
import os
import glob

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [23]:
df_test_pitch = pd.read_csv('../yakiu_rawdata/test_pitch.csv')
df_test_player = pd.read_csv('../yakiu_rawdata/test_player.csv')
df_train_pitch = pd.read_csv('../yakiu_rawdata/train_pitch.csv')
df_train_player = pd.read_csv('../yakiu_rawdata/train_player.csv')
df_sample = pd.read_csv('../yakiu_rawdata/sample_submit_ball_type.csv')

- testデータとTrainデータをマージしてから加工する。加工完了したのち分割する。

In [24]:
# test_pitchデータ加工。球種列のダミーを作成。
a = pd.DataFrame(np.nan, index = np.arange(len(df_test_pitch)), columns = ['球種'])
df_test_pitch = pd.concat([a,df_test_pitch],axis=1)

In [25]:
# 投球位置区域はもう一方のコンペでのみ使用のため、今回はdrop
df_train_pitch = df_train_pitch.drop('投球位置区域', axis=1)

In [26]:
# pitchデータをマージ
df_pitch_merge = pd.concat([df_train_pitch,df_test_pitch],axis=0)

In [51]:
# playerデータをマージ
df_player_merge = pd.concat([df_train_player,df_test_player],axis=0)

# プレイデータの加工

In [4]:
df_train_pitch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257117 entries, 0 to 257116
Data columns (total 51 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   データ内連番          257117 non-null  int64  
 1   球種              257117 non-null  int64  
 2   投球位置区域          257117 non-null  int64  
 3   年度              257117 non-null  int64  
 4   試合ID            257117 non-null  int64  
 5   試合内連番           257117 non-null  int64  
 6   試合内投球数          257117 non-null  int64  
 7   日付              257117 non-null  object 
 8   時刻              257117 non-null  object 
 9   ホームチームID        257117 non-null  int64  
 10  アウェイチームID       257117 non-null  int64  
 11  球場ID            257117 non-null  int64  
 12  球場名             257117 non-null  object 
 13  試合種別詳細          257117 non-null  object 
 14  イニング            257117 non-null  int64  
 15  表裏              257117 non-null  object 
 16  イニング内打席数        257117 non-null  int64  
 17  打席内投球数    

In [33]:
df_pitch =  df_pitch_merge.drop(['一塁手ID','二塁手ID','三塁手ID','遊撃手ID','左翼手ID','中堅手ID','左翼手ID'], axis=1)

In [34]:
df_pitch['count'] = df_pitch[['プレイ前ストライク数', 'プレイ前ボール数', 'プレイ前アウト数']].apply(lambda x: '{}-{}-{}'.format(x[0], x[1], x[2]), axis=1)

In [35]:
df_pitch['count']

0         0-0-0
1         1-0-0
2         2-0-0
3         0-0-1
4         1-0-1
          ...  
521645    2-1-1
521646    0-0-2
521647    1-0-2
521648    2-0-2
521649    2-1-2
Name: count, Length: 778767, dtype: object

In [36]:
h = df_pitch.query('表裏=="表"')
h['pitch_team_score'] = h['プレイ前アウェイチーム得点数']-h['プレイ前ホームチーム得点数']
a = df_pitch.query('表裏=="裏"')
a['pitch_team_score'] = a['プレイ前ホームチーム得点数']-a['プレイ前アウェイチーム得点数']
df_pitch = pd.concat([h,a],axis=0)
df_pitch =df_pitch.sort_values(['年度','データ内連番'])
df_pitch

C:\Users\bono_\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\bono_\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,データ内連番,球種,年度,試合ID,試合内連番,試合内投球数,日付,時刻,ホームチームID,アウェイチームID,球場ID,球場名,試合種別詳細,イニング,表裏,イニング内打席数,打席内投球数,投手ID,投手チームID,投手投球左右,投手役割,投手登板順,投手試合内対戦打者数,投手試合内投球数,投手イニング内投球数,打者ID,打者チームID,打者打席左右,打者打順,打者守備位置,打者試合内打席数,プレイ前ホームチーム得点数,プレイ前アウェイチーム得点数,プレイ前アウト数,プレイ前ボール数,プレイ前ストライク数,プレイ前走者状況,一塁走者ID,二塁走者ID,三塁走者ID,捕手ID,右翼手ID,成績対象投手ID,成績対象打者ID,count,pitch_team_score
0,0,0.0,2017,2017033101,1,1,2017-03-31,18:03:00,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,1,1,1500001,1,右,先発,1,1,1,1,900410,4,左,1,中,1,0,0,0,0,0,___,NaN,NaN,NaN,1300027,900397,1500001,900410,0-0-0,0
1,1,0.0,2017,2017033101,2,2,2017-03-31,18:03:12,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,1,2,1500001,1,右,先発,1,1,2,2,900410,4,左,1,中,1,0,0,0,0,1,___,NaN,NaN,NaN,1300027,900397,1500001,900410,1-0-0,0
2,2,0.0,2017,2017033101,3,3,2017-03-31,18:03:23,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,1,3,1500001,1,右,先発,1,1,3,3,900410,4,左,1,中,1,0,0,0,0,2,___,NaN,NaN,NaN,1300027,900397,1500001,900410,2-0-0,0
3,3,0.0,2017,2017033101,4,4,2017-03-31,18:04:01,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,2,1,1500001,1,右,先発,1,2,4,4,11436,4,右,2,二,1,0,0,1,0,0,___,NaN,NaN,NaN,1300027,900397,1500001,11436,0-0-1,0
4,4,0.0,2017,2017033101,5,5,2017-03-31,18:04:12,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,2,2,1500001,1,右,先発,1,2,5,5,11436,4,右,2,二,1,0,0,1,0,1,___,NaN,NaN,NaN,1300027,900397,1500001,11436,1-0-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521645,521645,NaN,2019,2019093001,289,285,2019-09-30,21:10:05,5,4,5,甲子園,セ・リーグ公式戦,9,表,3,8,11807,5,右,救援,6,3,17,17,700034,4,右,4,一,2,3,0,1,1,2,_2_,NaN,1700103.0,NaN,1300073,1500086,11807,700034,2-1-1,-3
521646,521646,NaN,2019,2019093001,290,286,2019-09-30,21:10:53,5,4,5,甲子園,セ・リーグ公式戦,9,表,4,1,11807,5,右,救援,6,4,18,18,1100069,4,左,5,三,4,3,0,2,0,0,_2_,NaN,1700103.0,NaN,1300073,1500086,11807,1100069,0-0-2,-3
521647,521647,NaN,2019,2019093001,291,287,2019-09-30,21:11:21,5,4,5,甲子園,セ・リーグ公式戦,9,表,4,2,11807,5,右,救援,6,4,19,19,1100069,4,左,5,三,4,3,0,2,0,1,_2_,NaN,1700103.0,NaN,1300073,1500086,11807,1100069,1-0-2,-3
521648,521648,NaN,2019,2019093001,292,288,2019-09-30,21:12:05,5,4,5,甲子園,セ・リーグ公式戦,9,表,4,3,11807,5,右,救援,6,4,20,20,1100069,4,左,5,三,4,3,0,2,0,2,_2_,NaN,1700103.0,NaN,1300073,1500086,11807,1100069,2-0-2,-3


In [37]:
df_date =df_pitch['日付'].str.split('-', expand=True)
df_date

,0,1,2
0,2017,03,31
1,2017,03,31
2,2017,03,31
3,2017,03,31
4,2017,03,31
...,...,...,...
521645,2019,09,30
521646,2019,09,30
521647,2019,09,30
521648,2019,09,30


In [38]:
df_date.rename(columns={0: 'year', 1: 'month',2:'day'}, inplace=True)
df_date = df_date.loc[:,['month']]
df_date = df_date.astype({'month':int})
df_date.head()

,month
0,3
1,3
2,3
3,3
4,3


In [39]:
df_pitch = pd.concat([df_pitch,df_date],axis=1)

In [40]:
df_pitch

,データ内連番,球種,年度,試合ID,試合内連番,試合内投球数,日付,時刻,ホームチームID,アウェイチームID,球場ID,球場名,試合種別詳細,イニング,表裏,イニング内打席数,打席内投球数,投手ID,投手チームID,投手投球左右,投手役割,投手登板順,投手試合内対戦打者数,投手試合内投球数,投手イニング内投球数,打者ID,打者チームID,打者打席左右,打者打順,打者守備位置,打者試合内打席数,プレイ前ホームチーム得点数,プレイ前アウェイチーム得点数,プレイ前アウト数,プレイ前ボール数,プレイ前ストライク数,プレイ前走者状況,一塁走者ID,二塁走者ID,三塁走者ID,捕手ID,右翼手ID,成績対象投手ID,成績対象打者ID,count,pitch_team_score,month
0,0,0.0,2017,2017033101,1,1,2017-03-31,18:03:00,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,1,1,1500001,1,右,先発,1,1,1,1,900410,4,左,1,中,1,0,0,0,0,0,___,NaN,NaN,NaN,1300027,900397,1500001,900410,0-0-0,0,3
1,1,0.0,2017,2017033101,2,2,2017-03-31,18:03:12,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,1,2,1500001,1,右,先発,1,1,2,2,900410,4,左,1,中,1,0,0,0,0,1,___,NaN,NaN,NaN,1300027,900397,1500001,900410,1-0-0,0,3
2,2,0.0,2017,2017033101,3,3,2017-03-31,18:03:23,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,1,3,1500001,1,右,先発,1,1,3,3,900410,4,左,1,中,1,0,0,0,0,2,___,NaN,NaN,NaN,1300027,900397,1500001,900410,2-0-0,0,3
3,3,0.0,2017,2017033101,4,4,2017-03-31,18:04:01,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,2,1,1500001,1,右,先発,1,2,4,4,11436,4,右,2,二,1,0,0,1,0,0,___,NaN,NaN,NaN,1300027,900397,1500001,11436,0-0-1,0,3
4,4,0.0,2017,2017033101,5,5,2017-03-31,18:04:12,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,2,2,1500001,1,右,先発,1,2,5,5,11436,4,右,2,二,1,0,0,1,0,1,___,NaN,NaN,NaN,1300027,900397,1500001,11436,1-0-1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521645,521645,NaN,2019,2019093001,289,285,2019-09-30,21:10:05,5,4,5,甲子園,セ・リーグ公式戦,9,表,3,8,11807,5,右,救援,6,3,17,17,700034,4,右,4,一,2,3,0,1,1,2,_2_,NaN,1700103.0,NaN,1300073,1500086,11807,700034,2-1-1,-3,9
521646,521646,NaN,2019,2019093001,290,286,2019-09-30,21:10:53,5,4,5,甲子園,セ・リーグ公式戦,9,表,4,1,11807,5,右,救援,6,4,18,18,1100069,4,左,5,三,4,3,0,2,0,0,_2_,NaN,1700103.0,NaN,1300073,1500086,11807,1100069,0-0-2,-3,9
521647,521647,NaN,2019,2019093001,291,287,2019-09-30,21:11:21,5,4,5,甲子園,セ・リーグ公式戦,9,表,4,2,11807,5,右,救援,6,4,19,19,1100069,4,左,5,三,4,3,0,2,0,1,_2_,NaN,1700103.0,NaN,1300073,1500086,11807,1100069,1-0-2,-3,9
521648,521648,NaN,2019,2019093001,292,288,2019-09-30,21:12:05,5,4,5,甲子園,セ・リーグ公式戦,9,表,4,3,11807,5,右,救援,6,4,20,20,1100069,4,左,5,三,4,3,0,2,0,2,_2_,NaN,1700103.0,NaN,1300073,1500086,11807,1100069,2-0-2,-3,9


In [43]:
df_pitch.dtypes

データ内連番                int64
球種                  float64
年度                    int64
試合ID                  int64
試合内連番                 int64
試合内投球数                int64
日付                   object
時刻                   object
ホームチームID              int64
アウェイチームID             int64
球場ID                  int64
球場名                  object
試合種別詳細               object
イニング                  int64
表裏                   object
イニング内打席数              int64
打席内投球数                int64
投手ID                  int64
投手チームID               int64
投手投球左右               object
投手役割                 object
投手登板順                 int64
投手試合内対戦打者数            int64
投手試合内投球数              int64
投手イニング内投球数            int64
打者ID                  int64
打者チームID               int64
打者打席左右               object
打者打順                  int64
打者守備位置               object
打者試合内打席数              int64
プレイ前ホームチーム得点数         int64
プレイ前アウェイチーム得点数        int64
プレイ前アウト数              int64
プレイ前ボール数              int64
プレイ前ストライク数          

In [47]:
# 実行しない
pitch_group = df_train_pitch3.groupby(['試合ID','イニング','表裏','イニング内打席数'])

In [48]:
# 実行しない
lag_pitch = pd.DataFrame(pitch_group['球種'].shift(1))

In [49]:
# 実行しない
lag_pitch = lag_pitch.fillna(15)
lag_pitch = lag_pitch.rename(columns={'球種': 'lag_pitch'})
lag_pitch['lag_pitch'] = lag_pitch['lag_pitch']
lag_pitch = lag_pitch.astype('int64')
lag_pitch

,lag_pitch
0,15
1,0
2,0
3,15
4,0
...,...
257112,15
257113,0
257114,0
257115,0


In [50]:
# 実行しない
df_train_pitch4 = pd.concat([df_train_pitch3,lag_pitch],axis=1)

In [57]:
# 実行しない
pd.to_pickle(df_train_pitch4,'./df_train_pitch4.pkl')

In [53]:
# 実行しない
df_train_pitch4 = pd.read_pickle('./df_train_pitch4.pkl')

In [54]:
# 実行しない
df_train_pitch4.to_csv('./df_train_pitch4.csv')

# playerファイルの加工

In [150]:
df_player = pd.DataFrame(df_player_merge)
df_player =df_player.reset_index(drop=True)

In [151]:
a = pd.DataFrame(range(0,len(df_age),1),columns=['No'])
df_player=pd.concat([a,df_player],axis=1)

In [152]:
df_player['経験年数'] = df_player['年度']-df_player['ドラフト年']

In [153]:
df_age = df_player['生年月日'].str.split('/',expand = True)
df_age = df_age.astype('int64')

# ここまで！！

In [149]:
print([df_age[1]<=3]

[0       False
1        True
2        True
3       False
4       False
        ...  
2752    False
2753    False
2754    False
2755     True
2756     True
Name: 1, Length: 2757, dtype: bool]


In [62]:
df_age.rename(columns={0:'birth'},inplace=True)
df_age =df_age.drop([1,2],axis=1)
df_age

,birth
0,1976
1,1981
2,1979
3,1980
4,1983
...,...
906,1998
907,1994
908,1997
909,1995


In [63]:
df_age.isnull().sum()

birth    0
dtype: int64

In [64]:
df_age = df_age.astype('int64')

In [65]:
df_train_player3 = pd.concat([df_train_player2,df_age],axis=1)

In [66]:
df_train_player3['age'] = df_train_player3['年度']-df_train_player3['birth']

In [67]:
df_train_player4 =df_train_player3.loc[:,['年度','選手ID','age']]

In [68]:
df_merge = pd.merge(df_train_pitch4,df_train_player4,left_on='投手ID',right_on='選手ID',how='left')

In [69]:
df_merge2 = df_merge.drop(['投球位置区域','試合ID','時刻','ホームチームID','アウェイチームID','球場ID','球場名','試合種別詳細','イニング内打席数','投手ID','投手チームID','打者ID','打者チームID','プレイ前ホームチーム得点数','プレイ前アウェイチーム得点数','一塁走者ID','二塁走者ID','三塁走者ID','捕手ID','成績対象投手ID','成績対象打者ID','年度','選手ID'],axis=1)

In [70]:
df_merge2

,データ内連番,球種,試合内連番,試合内投球数,日付,イニング,表裏,打席内投球数,投手投球左右,投手役割,投手登板順,投手試合内対戦打者数,投手試合内投球数,投手イニング内投球数,打者打席左右,打者打順,打者守備位置,打者試合内打席数,プレイ前アウト数,プレイ前ボール数,プレイ前ストライク数,プレイ前走者状況,右翼手ID,count,pitch_team_score,month,lag_pitch,age
0,0,0,1,1,2017-03-31,1,表,1,右,先発,1,1,1,1,左,1,中,1,0,0,0,___,900397,0-0-0,0,3,15,29
1,1,0,2,2,2017-03-31,1,表,2,右,先発,1,1,2,2,左,1,中,1,0,0,1,___,900397,1-0-0,0,3,0,29
2,2,0,3,3,2017-03-31,1,表,3,右,先発,1,1,3,3,左,1,中,1,0,0,2,___,900397,2-0-0,0,3,0,29
3,3,0,4,4,2017-03-31,1,表,1,右,先発,1,2,4,4,右,2,二,1,1,0,0,___,900397,0-0-1,0,3,15,29
4,4,0,5,5,2017-03-31,1,表,2,右,先発,1,2,5,5,右,2,二,1,1,0,1,___,900397,1-0-1,0,3,0,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257112,257112,0,293,288,2017-10-10,9,表,1,右,救援,6,7,20,13,右,7,捕,1,2,0,0,1__,1400170,0-0-2,-5,10,15,19
257113,257113,0,294,289,2017-10-10,9,表,2,右,救援,6,7,21,14,右,7,捕,1,2,0,1,1__,1400170,1-0-2,-5,10,0,19
257114,257114,0,295,290,2017-10-10,9,表,3,右,救援,6,7,22,15,右,7,捕,1,2,1,1,1__,1400170,1-1-2,-5,10,0,19
257115,257115,0,296,291,2017-10-10,9,表,4,右,救援,6,7,23,16,右,7,捕,1,2,1,2,1__,1400170,2-1-2,-5,10,0,19


In [71]:
pd.to_pickle(df_merge2,'df_merge2.pkl')